## 기본 신경망 구현

```
y = sigmoid(X x W + b)
y : 출력
Sigmoid : 활성화 함수
X : 입력
W : 가중치
b : 편향
```
### 학습이란 W, b의 값을 찾아내는 것을 말한다.

### 3-1 활성화 함수(activation function)

####  활성화 함수(activation function)는 인공 신경망을 통과해온 값을 최종적으로 어떤 값으로 만들지를 결정한다.
#### 종류 : Sigmoid(시그모이드), ReLu(렐루), tanh(쌍곡탄젠트)

* 인공뉴런은 가중치와 활성화 함수의 연결로 이루어진 간단한 구조이다.
* 간단한 개념의 인공 뉴런을 충분히 많이 연결된 것으로 인간이 인지하기 어려운 복잡한 패턴까지도 스스로 학습한다.

### 3-2 신경망 알고리즘 배경
 * 수천~수만개의 W와 b의 값을 일일이 변경시켜가며 계산하는데 오랜 시간이 걸림.
 * 신경망의 층이 깊어질 수록 시도해봐야 하는 경우가 수가 많아, 유의미한 신경망을 만들기가 어려웠음.
 * 제안 : 제프리 힌트(Geoffrey Hinton) 교수가 제한된 볼트만 머신(Restricted Boltzmann Machine, RBM)
 * 발전 : 드롭아웃 기법, ReLu 활성화 함수, GPU 발전, 역전파(backpropagation)

### 3-3 간단한 분류 모델 구현하기
* 딥러닝에서 가장 폭넓게 활용되는 분야는 패턴 인식을 통한 영상처리 분야이다.
* 패턴을 파악해 여러종류로 구분하는 작업을 분류(classification)이라 한다.

In [0]:
import tensorflow as tf
import numpy as np

### 데이터 정의 , 
* 털의 유무(1:있음, 0:없음)
* 날개의 유무(1:있음, 0:없음)


In [0]:
# [털, 날개]
# Tarket : 어떤 종류의 동물인지.
x_data = np.array( [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

In [0]:
# Tarket : 원핫인코딩(one-hot encoding)
# 데이터가 가질 수 있는 값들을 일렬로 나열한 배열로 만들고, 
# 그중의 표현하려는 값을 인덱스의 원소만 1로 표현하고, 나머지는 0으로 채우는 표기법

# [털, 날개]


# [기타, 포유류, 조류]
# 다음과 같은 형식을 one-hot 형식의 데이터라고 합니다.
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

### 신경망 모델 구성
* 플레이스 홀더 구성
* W : 특징수(2개), 레이블수(3개) 로 설정


In [0]:
X = tf.placeholder(tf.float32)   #  X에 들어갈 값(공간)
Y = tf.placeholder(tf.float32)   #  Y에 들어갈 값(공간)

# 신경망은 2차원으로 [입력층(특성), 출력층(레이블)] -> [2, 3] 으로 정합니다.
W = tf.Variable(tf.random_uniform([2,3], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정합니다.
# 편향은 아웃풋의 갯수, 즉 최종 결과값의 분류 갯수인 3으로 설정합니다.
b = tf.Variable(tf.zeros([3]))

* 01. 가중치를 곱하고 편향을 더한다. 
* 02. 활성화 함수를 적용한다.
* 03. 신경망을 통해 나온 출력값을 softmax 함수를 이용하여 사용하기 쉽게 다듬어준다.

In [0]:
L = tf.add(tf.matmul(X,W), b)

# 가중치와 편향을 이용해 계산한 결과 값에
# 텐서플로우에서 기본적으로 제공하는 활성화 함수인 ReLU 함수를 적용합니다.
L = tf.nn.relu(L)

# 마지막으로 softmax 함수를 이용하여 출력값을 사용하기 쉽게 만듭니다
# softmax 함수는 다음처럼 결과값을 전체합이 1인 확률로 만들어주는 함수입니다.
# 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
model = tf.nn.softmax(L)

### 손실함수(loss) 작성
 * 교차 엔트로피(Cross-Entropy)

In [0]:
# 신경망을 최적화하기 위한 비용 함수를 작성합니다.
# 각 개별 결과에 대한 합을 구한 뒤 평균을 내는 방식을 사용
# 전체 합이 아닌, 개별 결과를 구한 뒤 평균을 내는 방식을 사용하기 위해 axis 옵션을 사용합니다.
# axis 옵션이 없으면 -1.09 처럼 총합인 스칼라값으로 출력됩니다.
#        Y         model         Y * tf.log(model)   reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]
# 즉, 이것은 예측값과 실제값 사이의 확률 분포의 차이를 비용으로 계산한 것이며,
# 이것을 Cross-Entropy 라고 합니다.
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model), axis=1))
cost

<tf.Tensor 'Mean_5:0' shape=() dtype=float32>

### 학습 시키기

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)
print(train_op)

name: "GradientDescent_5"
op: "NoOp"
input: "^GradientDescent_5/update_Variable_12/ApplyGradientDescent"
input: "^GradientDescent_5/update_Variable_13/ApplyGradientDescent"



In [0]:
print(x_data.shape, y_data.shape)

(6, 2) (6, 3)


In [0]:
# 텐서플로 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 레이블 데이터를 이용하여 학습을 진행
for step in range(100):
  sess.run(train_op,feed_dict={X:x_data, Y:y_data})
  
  # 학습도중 10번씩 손실값을 출력
  if (step+1)%10 == 0:
    print(step+1, sess.run(cost, feed_dict={X:x_data, Y:y_data}))



10 1.0304931
20 1.0237601
30 1.0172738
40 1.0110192
50 1.0049826
60 0.9990556
70 0.99341303
80 0.98803854
90 0.9830895
100 0.97829556


### 새로운 코드

In [0]:
#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax 를 이용해 가장 큰 값을 가져옵니다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 2 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33
